In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import substring, col, when

In [2]:
#sc = SparkContext(appName="umsi618f22hw7")
sqlc = SQLContext(sc)

/sw/pkgs/arc/spark/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
tests=sqlc.read.option("header",True).option("inferschema",True).csv("/home/ruotongg/project1/tests.csv")
movies=sqlc.read.option("header",True).option("inferschema",True).csv("/home/ruotongg/project1/ml-25m/movies.csv")
ratings=sqlc.read.option("header",True).option("inferschema",True).csv("/home/ruotongg/project1/ml-25m/ratings.csv")
scores=sqlc.read.option("header",True).option("inferschema",True).csv("/home/ruotongg/project1/ml-25m/genome-scores.csv")
tags=sqlc.read.option("header",True).option("inferschema",True).csv("/home/ruotongg/project1/ml-25m/genome-tags.csv")

In [4]:
links=sqlc.read.option("header",True).option("inferschema",True).csv("/home/ruotongg/project1/ml-25m/links.csv")

In [5]:
tests.printSchema()

root
 |-- year: integer (nullable = true)
 |-- imdb: string (nullable = true)
 |-- title: string (nullable = true)
 |-- test: string (nullable = true)
 |-- clean_test: string (nullable = true)
 |-- binary: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- domgross: string (nullable = true)
 |-- intgross: string (nullable = true)
 |-- code: string (nullable = true)
 |-- budget_2013$: integer (nullable = true)
 |-- domgross_2013$: string (nullable = true)
 |-- intgross_2013$: string (nullable = true)
 |-- period code: integer (nullable = true)
 |-- decade code: integer (nullable = true)



In [6]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [7]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [8]:
scores.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- tagId: integer (nullable = true)
 |-- relevance: double (nullable = true)



In [9]:
tags.printSchema()

root
 |-- tagId: integer (nullable = true)
 |-- tag: string (nullable = true)



In [10]:
scores.show()

+-------+-----+--------------------+
|movieId|tagId|           relevance|
+-------+-----+--------------------+
|      1|    1|0.028749999999999998|
|      1|    2|0.023749999999999993|
|      1|    3|              0.0625|
|      1|    4| 0.07574999999999998|
|      1|    5|             0.14075|
|      1|    6|             0.14675|
|      1|    7|              0.0635|
|      1|    8|             0.20375|
|      1|    9|               0.202|
|      1|   10|             0.03075|
|      1|   11|             0.58025|
|      1|   12| 0.10249999999999998|
|      1|   13| 0.20174999999999998|
|      1|   14|0.007000000000000006|
|      1|   15|0.024500000000000022|
|      1|   16| 0.17275000000000001|
|      1|   17|0.016500000000000015|
|      1|   18| 0.10399999999999998|
|      1|   19|              0.6625|
|      1|   20| 0.30074999999999996|
+-------+-----+--------------------+
only showing top 20 rows



In [11]:
tests_new=tests.withColumn('imdb', substring('imdb', 3,8)).withColumn("imdb",col("imdb").cast("integer"))\
                .withColumn('binary', when(col('binary')=="PASS",1).otherwise(0)).withColumn("binary",col("binary").cast("integer"))

In [12]:
tests_new.show()

+----+-------+--------------------+----------------+----------+------+---------+---------+---------+--------+------------+--------------+--------------+-----------+-----------+
|year|   imdb|               title|            test|clean_test|binary|   budget| domgross| intgross|    code|budget_2013$|domgross_2013$|intgross_2013$|period code|decade code|
+----+-------+--------------------+----------------+----------+------+---------+---------+---------+--------+------------+--------------+--------------+-----------+-----------+
|2013|1711425|       21 &amp; Over|          notalk|    notalk|     0| 13000000| 25682380| 42195766|2013FAIL|    13000000|      25682380|      42195766|          1|          1|
|2012|1343727|            Dredd 3D|     ok-disagree|        ok|     1| 45000000| 13414714| 40868994|2012PASS|    45658735|      13611086|      41467257|          1|          1|
|2013|2024544|    12 Years a Slave| notalk-disagree|    notalk|     0| 20000000| 53107035|158607035|2013FAIL|    20

In [13]:
tests_new.printSchema()

root
 |-- year: integer (nullable = true)
 |-- imdb: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- test: string (nullable = true)
 |-- clean_test: string (nullable = true)
 |-- binary: integer (nullable = false)
 |-- budget: integer (nullable = true)
 |-- domgross: string (nullable = true)
 |-- intgross: string (nullable = true)
 |-- code: string (nullable = true)
 |-- budget_2013$: integer (nullable = true)
 |-- domgross_2013$: string (nullable = true)
 |-- intgross_2013$: string (nullable = true)
 |-- period code: integer (nullable = true)
 |-- decade code: integer (nullable = true)



In [14]:
tests_new.createOrReplaceTempView("tests")
movies.createOrReplaceTempView("movies")
ratings.createOrReplaceTempView("ratings")
scores.createOrReplaceTempView("scores")
tags.createOrReplaceTempView("tags")
links.createOrReplaceTempView("links")

In [15]:
base=sqlc.sql("""SELECT ratings.movieId, tests.title, tests.binary, AVG(ratings.rating) AS avg_rating, COUNT(ratings.rating) AS rating_num
    FROM ratings
    INNER JOIN (SELECT movieId, imdbId
    FROM links) AS temp
    ON ratings.movieId=temp.movieId 
    INNER JOIN tests
    ON temp.imdbId=tests.imdb
    GROUP BY ratings.movieId, tests.title, tests.binary
    """)

In [16]:
base.show()

+-------+--------------------+------+------------------+----------+
|movieId|               title|binary|        avg_rating|rating_num|
+-------+--------------------+------+------------------+----------+
|   3897|       Almost Famous|     1|3.8336601800170964|     19887|
|   1566|            Hercules|     1|3.4190598656950995|      6999|
|  64957|The Curious Case ...|     0|3.5953075042317097|     10634|
|  89745|        The Avengers|     0| 3.780246983837924|     17572|
|  61236|   Waltz with Bashir|     0| 3.887579617834395|      1570|
|  66171|                Push|     1|3.2139406487232574|      1449|
|  88356|          The Smurfs|     1|2.4280575539568345|       556|
|   2617|           The Mummy|     0|3.2055174579908914|     19103|
|   3175|        Galaxy Quest|     0|3.6077836141619484|     14659|
|  50685|            Waitress|     1|3.5284728213977568|      1159|
|  58047|   Definitely, Maybe|     1|3.5668953687821614|      2332|
|     57|Home for the holi...|     1| 3.32043828

In [17]:
base.createOrReplaceTempView("base")

In [18]:
q1_1=sqlc.sql("""SELECT *
    FROM base
    ORDER BY avg_rating DESC
    LIMIT 30
    """)

In [19]:
q1_1.show()

+-------+--------------------+------+------------------+----------+
|movieId|               title|binary|        avg_rating|rating_num|
+-------+--------------------+------+------------------+----------+
|    318|The Shawshank Red...|     0| 4.413576004516335|     81482|
|    858|       The Godfather|     0| 4.324336165187245|     52498|
|     50|  The Usual Suspects|     0| 4.284353213163313|     55366|
|   1221|The Godfather: Pa...|     1|4.2617585117585115|     34188|
|    527|Schindler&#39;s List|     1| 4.247579083279535|     60411|
|   2959|          Fight Club|     0| 4.228310618821568|     58773|
|   1193|One Flew Over the...|     0|4.2186616007543405|     36058|
|  44555|Das Leben Der And...|     0| 4.200392285060477|      9177|
|    296|        Pulp Fiction|     1| 4.188912039361382|     79672|
|   1213|          Goodfellas|     0|4.1805253650920005|     32663|
|  58559|     The Dark Knight|     0| 4.166538211421277|     41519|
|  79132|           Inception|     0| 4.15550842

In [20]:
q1_1.coalesce(1).write.option("delimiter", "\t").option("header", "true").csv("ruotongg_si618_prj1_output_1_1")

In [21]:
q1_2=sqlc.sql("""SELECT *
    FROM base
    ORDER BY rating_num DESC
    LIMIT 30
    """)

In [22]:
q1_2.show()

+-------+--------------------+------+------------------+----------+
|movieId|               title|binary|        avg_rating|rating_num|
+-------+--------------------+------+------------------+----------+
|    356|        Forrest Gump|     0| 4.048011436845787|     81491|
|    318|The Shawshank Red...|     0| 4.413576004516335|     81482|
|    296|        Pulp Fiction|     1| 4.188912039361382|     79672|
|    593|The Silence of th...|     1| 4.151341616415071|     74127|
|   2571|          The Matrix|     1| 4.154099127610975|     72674|
|    260|           Star Wars|     0| 4.120188599618726|     68717|
|    480|       Jurassic Park|     1|3.6791749812920926|     64144|
|    527|Schindler&#39;s List|     1| 4.247579083279535|     60411|
|    110|          Braveheart|     0| 4.002272573668559|     59184|
|   2959|          Fight Club|     0| 4.228310618821568|     58773|
|    589|Terminator 2: Jud...|     0|  3.94637410899458|     57379|
|   1196|Star Wars: Episod...|     0| 4.14412231

In [23]:
q1_2.coalesce(1).write.option("delimiter", "\t").option("header", "true").csv("ruotongg_si618_prj1_output_1_2")

In [24]:
q1_3=sqlc.sql("""SELECT COUNT(binary), SUM(binary)
    FROM base
    """)

In [25]:
q1_3.show()

+-------------+-----------+
|count(binary)|sum(binary)|
+-------------+-----------+
|         1787|        799|
+-------------+-----------+



In [26]:
799/1787

0.4471180749860101

In [27]:
base_tag=sqlc.sql("""SELECT AVG(base.binary) AS bechedel, temp.tag, COUNT(base.binary) AS tag_number
    FROM base
    INNER JOIN (SELECT scores.movieId, scores.tagId, scores.relevance, tags.tag
    FROM scores
    INNER JOIN tags
    ON scores.tagId=tags.tagId
    WHERE scores.relevance >=0.05) AS temp
    ON base.movieId=temp.movieId
    GROUP BY temp.tag
    HAVING tag_number>=5
    ORDER BY bechedel DESC
    """)

In [28]:
base_tag.show()

+------------------+--------------------+----------+
|          bechedel|                 tag|tag_number|
+------------------+--------------------+----------+
|0.8571428571428571|              ballet|        21|
|            0.8125|           almodovar|        16|
|               0.8|      beauty pageant|       190|
|        0.74609375|mother daughter r...|       256|
|0.7170172084130019|             sisters|       523|
|0.7073170731707317|         jane austen|       123|
|          0.703125|            abortion|        64|
|0.6985645933014354|     boarding school|       209|
|0.6976744186046512|               opera|        43|
|0.6896551724137931|             cooking|       116|
|0.6882793017456359|             heroine|       401|
|0.6674157303370787|           pregnancy|       445|
| 0.659942363112392|             lesbian|       347|
|0.6491228070175439|         transgender|       228|
|0.6458333333333334|         emma watson|        48|
|0.6427406199021207|           teenagers|     

In [29]:
base_tag.coalesce(1).write.option("delimiter", "\t").option("header", "true").csv("ruotongg_si618_prj1_output_2")

In [30]:
prefer=sqlc.sql("""SELECT ratings.userId, AVG(base.binary*ratings.rating) AS avg_preference, STD(base.binary*ratings.rating) AS std_preference
    FROM base
    INNER JOIN ratings
    ON base.movieId=ratings.movieId
    GROUP BY ratings.userId
    """)

In [31]:
prefer.show()

+------+------------------+------------------+
|userId|    avg_preference|    std_preference|
+------+------------------+------------------+
|   833|1.2293577981651376|1.7853987391706492|
|  1088|               0.7|1.5952515745821712|
|  1238|0.9857142857142858| 1.835097386402808|
|  1342|1.0078740157480315|1.7468031240027329|
|  1580| 2.727272727272727| 2.195035721390843|
|  1829|1.6333333333333333|1.9685733155959433|
|  1959|1.1611111111111112| 1.904795512009566|
|  3749|1.7241379310344827| 1.887866364144188|
|  3794|0.8695652173913043|1.5484700951152113|
|  3918|0.9299516908212561|1.5252475519541977|
|  3997|0.8076923076923077|1.2332298669180541|
|  4900| 1.090128755364807|1.6733294501884084|
|  5803|1.0246478873239437|1.7507115724439368|
|  6336|0.6274509803921569|1.3558876695910393|
|  6357|              1.59| 2.131272679016749|
|  6466| 1.411764705882353| 1.818532051829005|
|  6658|1.0116279069767442|1.7944636156172369|
|  7754|           0.15625|0.5148989879014481|
|  7880| 1.03

In [32]:
prefer.coalesce(1).write.option("delimiter", "\t").option("header", "true").csv("ruotongg_si618_prj1_output_3")